In [50]:
import rqdatac
from rqdatac import *

import datetime
import pandas as pd
import numpy as np
from pandas.tseries.offsets import *

import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot

In [51]:
rqdatac.init('xinjin', '123456', ('172.19.182.162', 16003))

In [52]:
get_next_trading_date('2017-1-1')

datetime.date(2017, 1, 3)

In [53]:
# initialize plotly
init_notebook_mode(connected=True)

In [58]:
## helper functions
# keep only year and moth(year-month) as string of datetime object
# return example: 2017-01-01
def date2ym_str(date):
    y = date.year
    m = date.month
    ym = '{}-{}'.format(y, m)
    return ym

def date2ymd_str(date):
    y = date.year
    m = date.month
    d = date.day
    ymd = '{}-{}-{}'.format(y, m, d)
    return ymd

def datetime2date(date_time):
    y = date_time.year
    m = date_time.month
    d = date_time.day
    return datetime.date(y, m, d)

def date2datetime(date):
    y = date.year
    m = date.month
    d = date.day
    return datetime.datetime(y, m, d)

def adjust_to_trading_date(date_time, trading_dates_list):
    ''' trading_dates_list is a list of string indicate date
    '''
    ymd_str = date2ymd_str(date_time)

    if ymd_str in trading_dates_list:    # this date is trading date
        if date_time.hour >= 15:    # event should be in next day
            return get_next_trading_date(ymd_str)
        else: # return date as datetime.date() type
            return datetime2date(date_time)
    else: # this date is not trading day, return next trading day
        return get_next_trading_date(ymd_str)

In [72]:
def filter_title(title):
    # word must be in title
    contain = False
    if '增持' in title:
        contain = True
        if '完成' in title:    # if contain both words, must be right
            return True

    # words that should not be in title
    exclude_list = ['误操作', '倡议书', '进展', '核查意见', '补充', '计划' ]
    for w in exclude_list:
        if w in title:
            return False

    return contain

def complete_code(code):

    if len(code) < 6: # code is empty or length smaller than 6
        return False
    elif code[:3] in ['600', '601']: # 上证
        return code + '.XSHG'
    elif code[:3] == '000': # 深证
        return code + '.XSHE'
    else: # neither
        return False

def announce2event(file_name, backtest_start_date=None):
    '''
        @param file_name is the csv file of announcements to read from
        @param date is the start date of backtest
    '''

    # read csv file into dataframe
    df = pd.read_csv(file_name, dtype=str,
                    parse_dates=True,
                    index_col='Date',
                    usecols=['Code', 'Title', 'Link', 'Date'],
                    na_values=['nan'])
    
#     index = df.index
#     print(index)
#     new_index = index
#     for d in index:
#         print(d)
#         new_index.append(''.join(datetime.datetime.strptime(' '.join(str(index).split()), '%Y-%m-%d %H:%M:%S')))
#     df = df.reindex(new_index)
    #################################### check ###################################
    print(df[-20:])
    print(type(df.index[0]))
    print(df.index[0])
#     print(df.index[0] > date2datetime(datetime.date(2017, 6, 12)))
    print(df.columns)
    # check index type
    for i in df.index:
        if i == '':
            print("There are null in df index!!!")
        elif not isinstance(i, type(datetime.datetime(2017,1,1))):
            print("There are {} type in df.index!!!".format(type(i)))
        else:
            pass
    #################################### check ###################################
    
    if backtest_start_date != None:
        try:
            # slice rows with date after backtest start date
            df = df[df.index > date2datetime(backtest_start_date)-BDay()]
        except:
            print('Something wrong with slice date of event df')

    # get date range
    start_date = date2ymd_str(df.index[-1])
    end_date = date2ymd_str(df.index[0])
    # get all valid trading dates
    trading_dates = get_trading_dates(start_date, end_date)
    # event df, no need to construct index and columns name, it's constructed on the fly
    event_df = pd.DataFrame(index=trading_dates)

    # loop over rows of df
    for date, row in df.iterrows():
        code = complete_code(str(row['Code']))
        # code has meaning and title pass the filter
        if date and code and filter_title(row['Title']):
#             try:
                # keep only year-month-day, convert to datetime, index is list of trading dates 
                event_df.loc[adjust_to_trading_date(date, trading_dates), code] = 1
#             except:
#                 print('Set event error at :{}, code :{}'.format(date, code))
    
    # NOTICE: Remember to reverse the order of row because date index in 
    # csv file is in descending order, while date index in event_df should be
    # in ascending order.
#     event_df = event_df.iloc[::-1]

    return event_df

In [89]:
# there are 4,307,969 announcements in file all.csv
# event_df = announce2event('announcements_abstract.csv', datetime.date(2010, 1, 1))
event_df = announce2event('announcements_abstract_2.csv')
print('Totally {} events'.format(event_df.sum().sum()))

              Code                                       Title  \
Date                                                             
2015-06-30  124072                       2014年发行人履约情况及偿债能力分析报告   
2015-06-30  124061                     大公关于延迟披露2015年度跟踪评级报告的公告   
2015-06-30  124040                     大公关于延迟披露2015年度跟踪评级报告的公告   
2015-06-30  124038                    关于2014年度发行人履约情况及偿债能力分析报告   
2015-06-30  124029                     大公关于延迟披露2015年度跟踪评级报告的公告   
2015-06-30  123075              东兴证券股份有限公司2015年第一期次级债券非公开转让告知书   
2015-06-30  123063              中国国际金融有限公司2015年第一期次级债券非公开转让告知书   
2015-06-30  123012                     大公关于延迟披露2015年度跟踪评级报告的公告   
2015-06-30  122937                     大公关于延迟披露2015年度跟踪评级报告的公告   
2015-06-30  122894                      关于延迟披露2015年定期跟踪评级报告的公告   
2015-06-30  122893                     大公关于延迟披露2015年度跟踪评级报告的公告   
2015-06-30  122870  中诚信国际关于延迟披露重庆大晟资产经营（集团）有限公司2015年度跟踪评级报告的公告   
2015-06-30  122832                      关于延迟披露2015年定期跟踪评级报告的公告   
2015-06-30

In [90]:
print(event_df)

            600556.XSHG  600615.XSHG  000722.XSHE  600860.XSHG  601111.XSHG  \
2015-06-30          NaN          NaN          NaN          NaN          NaN   
2015-07-01          NaN          NaN          NaN          NaN          NaN   
2015-07-02          NaN          NaN          NaN          NaN          NaN   
2015-07-03          NaN          NaN          NaN          NaN          NaN   
2015-07-06          NaN          NaN          NaN          NaN          NaN   
2015-07-07          NaN          NaN          NaN          NaN          NaN   
2015-07-08          NaN          NaN          NaN          NaN          NaN   
2015-07-09          NaN          NaN          NaN          NaN          NaN   
2015-07-10          NaN          NaN          NaN          NaN          NaN   
2015-07-13          NaN          NaN          NaN          1.0          NaN   
2015-07-14          NaN          NaN          NaN          NaN          NaN   
2015-07-15          NaN          NaN          NaN   

In [91]:
## Event class
class Event:
    """Back test performance based on rate of return chosen by a specific event
    
    :param event_df: DataFrame of events, stock on column, date on row
    """

    def __init__(self, event_df):
        print("Generating event instance...")
        # initialize vars
        self.event_df = event_df # date in row, stock index in column
        self.event_list = list()
        self.price_df = event_df * np.nan
        self.absolute_performance = pd.DataFrame()
        self.relative_performance = pd.DataFrame()
        # get data
        self.get_prices()
        self.get_event_list()
        print("")
    
    # construct list of events from event_df        
    def get_event_list(self):
        # build a list of tuples (s_index, e_date)
        print("Generating event_list...")
        # loop over rows of event_df
        for event_date, row in self.event_df.iterrows():
            # NOTICE: event_df has elements of type 'Object', which are pointers to objects, has to get object itself first
            row_drop = row[np.isfinite(row.values.astype(np.float64))] # drop all NaN, left only events
            # loop over every element in row series
            for stock_index, event_indicator in row_drop.iteritems(): # event_indicator is always 1
                self.event_list.append((stock_index, event_date))
        print('Done! Found totally {} events.'.format(len(self.event_list)))
        
    def get_prices(self, fields='close', adjust_type='post', frequency='1d'):
        # collect prices needed
        print("Collecting price data...")
        # self.events.df.columns contains stock codes
        stock_list = list(self.event_df.columns)
        # self.events.df.index contains dates we need
        s_date = self.event_df.index[0] - 20*BDay()
        e_date = self.event_df.index[-1] + 20*BDay()
        
        self.price_df = get_price(stock_list,
                                  start_date=s_date,
                                  end_date=e_date,
                                  fields=fields,
                                  adjust_type=adjust_type,
                                  frequency=frequency)
        

        
    def get_absolute_performance(self, lookforward_num, lookbackward_num=0, boll_std_num=1):
        """Compute and plot absolute rate of return
        
        :param lookforward_num: Number of days in concern in the future
        :param lookforward_num: Number of days in concern in the past, if not specified, ignore
        """
        print("Calculating absolute performance")
        # rows are events, columns are days
        absolute_performance_list_forward = list()
        absolute_performance_list_backward = list()
        
        # iterate to analyze every event in self.events_list
        for e in self.event_list:
            # event data format: tuple(code_of_event_stock, date_of_the_event)
            stock = e[0]
            date = e[1]
            # get location number of current date
            event_row = self.price_df.index.get_loc(date)
            forward_row = min(event_row + lookforward_num, len(self.price_df))
            if lookbackward_num > 0: 
                backward_row = max(event_row - lookbackward_num, 0)

            try: # deal with lookforward
                # slice prices during target period
                price_forward = self.price_df.ix[event_row:(forward_row+1), stock]
                # transform price to net value, nv = 1 + accumulated return rate, then minus 1
                net_value_forward = (price_forward / price_forward[0]).values - 1
                # append nv for this event to the total performance list
                absolute_performance_list_forward.append(net_value_forward)
            except:
                # print relavent variables for debugging purpose
                print('slice price error at event row: {}'.format(event_row))
                print('slice price error at forward row: {}'.format(forward_row))
            
            if lookbackward_num > 0: # deal with lookbackward
                try:  
                    price_backward = self.price_df.ix[backward_row:(event_row+1), stock]
                    # reverse the order to align event_row, [event_day, event_day-1, ...]
                    price_backward = price_backward.sort_index(ascending=False)
                    # transform price to net value, nv = 0 + accumulated returns
                    net_value_backward = (price_backward / price_backward[0]).values - 1
                    absolute_performance_list_backward.append(net_value_backward)
                except:
                    print('slice price error at event row: {}'.format(event_row))
                    print('slice price error at backward row: {}'.format(backward_row))
        
        # transform list of arrays to DataFrame
        absolute_performance_df_forward = pd.DataFrame(absolute_performance_list_forward)
        
        if lookbackward_num > 0:
            # create and then reverse order of backward dataframe [event_day-n, event_day-n-1, ..., event_day]
            absolute_performance_df_backward = pd.DataFrame(absolute_performance_list_backward)
            absolute_performance_df_backward = absolute_performance_df_backward.sort_index(axis=1, ascending=False)
            # rename columns to negative day representation
            columns_backward = range(1-absolute_performance_df_backward.shape[1], 1)
            absolute_performance_df_backward.columns = columns_backward
            # concat forward and backward dataframe, drop one of repeated column 0, which is event day
            self.absolute_performance = pd.concat([absolute_performance_df_backward.drop(0, axis=1), 
                                                   absolute_performance_df_forward], axis=1)
        else:
            self.absolute_performance = absolute_performance_df_forward
        
        ## Plot
        # plot absolute performance
        self.plot_band(self.absolute_performance.mean(), title_str='Absolute Performance', yaxis_str='Rate of Return')
        
        # plot win rate
        win_rate = self.absolute_performance.copy( )
        # mark value bigger than 0 as 1, others 0
        win_rate[win_rate > 0] = 1
        win_rate[win_rate <= 0] = 0
        # winning count divide by total number of events with respect to specific days
        day_count = win_rate.shape[0]
        win_rate = win_rate.sum(axis=0) / day_count
        
        self.plot_area(win_rate, title_str='Win Rate (Absolute)')
        
        
    def get_relative_performance(self, benchmark_index, lookforward_num, lookbackward_num=0, boll_std_num=1):
        """Compute and plot relative rate of return, long-short portfolio
        
        :param benchmark_index: Benchmark to compare with
        :param lookforward_num: Number of days in concern in the future
        :param lookforward_num: Number of days in concern in the past, if not specified, ignore
        """
        print("Calculating relative performance")
        # collect benchmark index prices
        s_date = self.event_df.index[0] - 20*BDay()
        e_date = self.event_df.index[-1] + 20*BDay()
        fld = 'close'
        adj = 'post'
        frq= '1d'
        benchmark_price = get_price(benchmark_index,
                                    start_date=s_date,
                                    end_date=e_date,
                                    fields=fld,
                                    adjust_type=adj,
                                    frequency=frq)
        
        relative_performance_list_forward = list()
        relative_performance_list_backward = list()

        for e in self.event_list:
            stock = e[0]
            date = e[1]
            # get location number of current date
            event_row = self.price_df.index.get_loc(date)
            forward_row = min(event_row + lookforward_num, len(self.price_df))
            if lookbackward_num > 0:
                backward_row = max(event_row - lookbackward_num, 0)

            try: # deal with forward
                # slice prices during target period
                price_forward = self.price_df.ix[event_row:forward_row, stock]
                price_bench_forward = benchmark_price.ix[event_row:forward_row]
                # transform price to net value, nv = 1 + accumulated returns
                net_value_forward = (price_forward / price_forward[0]).values
                net_value_bench_forward = (price_bench_forward / price_bench_forward[0]).values
                # calculate and append relative performance for this event to the total performance list
                # NOTICE:
                # Unlike net value, relative_performance = 0 + accumulated long-short portfolio returns,
                # since both nv and nv_benchmark contains 1 and their cumrets(accumulated returns).
                relative_performance_forward = net_value_forward - net_value_bench_forward
                relative_performance_list_forward.append(relative_performance_forward)
            except:
                # print relavent variables for debugging purpose
                print('slice price error at event row: {}'.format(event_row))
                print('slice price error at forward row: {}'.format(forward_row))
            
            if lookbackward_num > 0: # deal with backward
                try:
                    # slice prices during target period
                    price_backward = self.price_df.ix[backward_row:(event_row+1), stock]
                    price_bench_backward = benchmark_price.ix[backward_row:(event_row+1)]
                    # reverse the order to aline event_row, [event_day, event_day-1, ...]
                    price_backward = price_backward.sort_index(ascending=False)
                    price_bench_backward = price_bench_backward.sort_index(ascending=False)
                    # transform price to net value, nv = 1 + accumulated returns
                    net_value_backward = (price_backward / price_backward[0]).values
                    net_value_bench_backward = (price_bench_backward / price_bench_backward[0]).values
                    # calculate and append relative performance for this event to the total performance list
                    # NOTICE:
                    # Unlike net value, relative_performance = 0 + accumulated long-short portfolio returns,
                    # since both nv and nv_benchmark contains 1 and their cumrets(accumulated returns).
                    relative_performance_backward = net_value_backward - net_value_bench_backward
                    relative_performance_list_backward.append(relative_performance_backward)
                except:
                    # print relavent variables for debugging purpose
                    print('slice price error at event row: {}'.format(event_row))
                    print('slice price error at backward row: {}'.format(backward_row))

        # transform list of arrays to DataFrame        
        relative_performance_df_forward = pd.DataFrame(relative_performance_list_forward)

        if lookbackward_num > 0:
            # create and then reverse order of backward dataframe [event_day-n, event_day-n-1, ..., event_day]
            relative_performance_df_backward = pd.DataFrame(relative_performance_list_backward)
            relative_performance_df_backward = relative_performance_df_backward.sort_index(axis=1, ascending=False)
            # rename columns to negative day representation
            columns_backward = range(1-relative_performance_df_backward.shape[1], 1)
            relative_performance_df_backward.columns = columns_backward
            # concat forward and backward dataframe, drop one of repeated column 0, which is event day
            self.relative_performance = pd.concat([relative_performance_df_backward.drop(0, axis=1), 
                                                   relative_performance_df_forward], axis=1)
        else:
            self.relative_performance = relative_performance_df_forward
        
        ## Plot
        self.plot_band(self.relative_performance.mean(), title_str='Relative Performance', yaxis_str='Rate of Return')
        
        # plot win rate
        win_rate = self.relative_performance.copy( )
        # mark value bigger than 0 as 1, others 0
        win_rate[win_rate > 0] = 1
        win_rate[win_rate <= 0] = 0
        # winning count divide by total number of events with respect to specific days
        day_count = win_rate.shape[0]
        win_rate = win_rate.sum(axis=0) / day_count
        
        self.plot_area(win_rate, title_str='Win Rate (Relative)')
    
    def event_distribution(self, month=True):
        # plot event distribution by month or by day, depend on parameter 'month'
        print('Plotting event distribution...')
        if month:   # count by month   
            # group index by month, then sum over month, need to convert index to datetime first!
            event_df_month = self.event_df.copy()
            event_df_month.index = pd.to_datetime(event_df_month.index)
            event_df_month = event_df_month.resample('M').sum()
            # sum over column
            event_count_month = event_df_month.sum(axis=1)
            # modify index, keep only year and month
            index_new = pd.Series(event_count_month.index)
            index_new = index_new.apply(date2ym_str)
            event_count_month.index = index_new
            # plot
            self.plot_bar(event_count_month, title_str='Event Distribution By Month')

        else:   # count by day
            # sum over column
            event_count_day = self.event_df.sum(axis=1)
            # plot
            self.plot_bar(event_count_day, title_str='Event Distribution By Day')
            
    
    # plot a time series and a band deviate by std_num of std
    def plot_band(self, time_series, title_str, yaxis_str, std_num=1):
        # # sign in
        # py.sign_in('hyqLeonardo', 'aHHAi8RbFuit2fOfEizB')

        mean = time_series
        std = mean.std()
        upper = mean + std_num * std
        lower = mean - std_num * std

        upper_bound = go.Scatter(
            name='Upper Bound',
            x=mean.index,
            y=upper,
            mode='lines',
            marker=dict(color="444"),
            line=dict(width=0),
            fillcolor='rgba(68, 68, 68, 0.3)',
            fill='tonexty' )

        trace = go.Scatter(
            name='Measurement',
            x=mean.index,
            y=mean,
            mode='lines',
            line=dict(color='rgb(31, 119, 180)'),
            fillcolor='rgba(68, 68, 68, 0.3)',
            fill='tonexty' )

        lower_bound = go.Scatter(
            name='Lower Bound',
            x=mean.index,
            y=lower,
            marker=dict(color="444"),
            line=dict(width=0),
            mode='lines' )

        data = [lower_bound, trace, upper_bound]

        layout = go.Layout(
            paper_bgcolor='rgba(0,0,0,0)',
            yaxis=dict(title=yaxis_str),
            title=title_str,
            showlegend = False)

        fig = go.Figure(data=data, layout=layout)

        iplot(fig, filename=title_str)
    
    # plot line and area beneath of a time series
    def plot_area(self, time_series, title_str):
        trace = go.Scatter(
            x=time_series.index,
            y=time_series,
            fill='tonexty'
        )
        
        layout = go.Layout(
            paper_bgcolor='rgba(0,0,0,0)',
            title=title_str,
            showlegend = False
        )
        
        data = [trace]
        fig = go.Figure(data=data, layout=layout)
        
        iplot(fig, filename=title_str)
        
    # plot bar chart of a time sereis
    def plot_bar(self, time_series, title_str):
        bar = [go.Bar(
                    x=time_series.index,
                    y=time_series
            )]
        
        layout = go.Layout(
            paper_bgcolor='rgba(0,0,0,0)',
            title=title_str,
            showlegend = False
        )
        
        data = bar
        fig = go.Figure(data=data, layout=layout)
        
        iplot(data, filename=title_str)

In [81]:
E = Event(event_df)
print(E.event_list)

Generating event instance...
Generating event_list...
Done! Found totally 672 events.

[('000100.XSHE', datetime.date(2009, 4, 27)), ('600503.XSHG', datetime.date(2009, 5, 12)), ('600406.XSHG', datetime.date(2009, 5, 15)), ('600690.XSHG', datetime.date(2009, 5, 25)), ('600872.XSHG', datetime.date(2009, 5, 25)), ('600249.XSHG', datetime.date(2009, 6, 8)), ('600536.XSHG', datetime.date(2009, 6, 9)), ('000009.XSHE', datetime.date(2009, 6, 12)), ('000040.XSHE', datetime.date(2009, 6, 12)), ('000982.XSHE', datetime.date(2009, 6, 19)), ('000540.XSHE', datetime.date(2009, 6, 24)), ('600690.XSHG', datetime.date(2009, 6, 26)), ('000823.XSHE', datetime.date(2009, 7, 8)), ('600066.XSHG', datetime.date(2009, 7, 9)), ('600697.XSHG', datetime.date(2009, 7, 22)), ('000761.XSHE', datetime.date(2009, 7, 22)), ('600697.XSHG', datetime.date(2009, 7, 24)), ('600712.XSHG', datetime.date(2009, 7, 27)), ('601168.XSHG', datetime.date(2009, 7, 30)), ('600865.XSHG', datetime.date(2009, 8, 17)), ('601111.XSHG', 

In [82]:
E.event_distribution(month=True)

Plotting event distribution...


In [92]:
E.get_absolute_performance(30, 30)

Calculating absolute performance


In [93]:
E.get_relative_performance('000300.XSHG', 30, 30)

Calculating relative performance
